<a href="https://colab.research.google.com/github/Kondojupradeep/OBSIP/blob/main/Autocomplete_and_Autocorrectness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv("/content/creditcard.csv")

In [ ]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87240 entries, 0 to 87239
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    87240 non-null  int64  
 1   V1      87240 non-null  float64
 2   V2      87240 non-null  float64
 3   V3      87240 non-null  float64
 4   V4      87240 non-null  float64
 5   V5      87239 non-null  float64
 6   V6      87239 non-null  float64
 7   V7      87239 non-null  float64
 8   V8      87239 non-null  float64
 9   V9      87239 non-null  float64
 10  V10     87239 non-null  float64
 11  V11     87239 non-null  float64
 12  V12     87239 non-null  float64
 13  V13     87239 non-null  float64
 14  V14     87239 non-null  float64
 15  V15     87239 non-null  float64
 16  V16     87239 non-null  float64
 17  V17     87239 non-null  float64
 18  V18     87239 non-null  float64
 19  V19     87239 non-null  float64
 20  V20     87239 non-null  float64
 21  V21     87239 non-null  float64
 22

In [ ]:
data.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        1
V6        1
V7        1
V8        1
V9        1
V10       1
V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [ ]:
data['Class'].value_counts()

Class
0.0    87034
1.0      205
Name: count, dtype: int64

In [ ]:
# Seperate data for analysis
legit = data[data.Class ==0]
fraud = data[data.Class ==1]


print(legit.shape)
print(fraud.shape)

(87034, 31)
(205, 31)


In [ ]:
# Statistical measures of the data

legit.Amount.describe()

count    87034.000000
mean        98.392549
std        267.475633
min          0.000000
25%          7.680000
50%         27.000000
75%         89.790000
max      19656.530000
Name: Amount, dtype: float64

In [ ]:
#Compare the values for both transaction
data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,39219.680136,-0.249921,-0.050180,0.700945,0.151490,-0.267417,0.099006,-0.094600,0.047273,-0.018269,...,0.040984,-0.031313,-0.105935,-0.036580,0.009933,0.132438,0.026564,-0.000518,0.002005,98.392549
1.0,34268.526829,-6.385940,4.394241,-8.432887,5.084061,-4.597691,-1.955100,-6.705734,2.927156,-3.042642,...,0.389325,0.752293,-0.146696,-0.238003,-0.095301,0.223491,0.093696,0.560829,0.044682,101.984439


In [ ]:
legit_sample = legit.sample(n=492)

In [ ]:
newdata = pd.concat([legit_sample, fraud], axis=0)

newdata.info()

<class 'pandas.core.frame.DataFrame'>
Index: 697 entries, 7904 to 86155
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    697 non-null    int64  
 1   V1      697 non-null    float64
 2   V2      697 non-null    float64
 3   V3      697 non-null    float64
 4   V4      697 non-null    float64
 5   V5      697 non-null    float64
 6   V6      697 non-null    float64
 7   V7      697 non-null    float64
 8   V8      697 non-null    float64
 9   V9      697 non-null    float64
 10  V10     697 non-null    float64
 11  V11     697 non-null    float64
 12  V12     697 non-null    float64
 13  V13     697 non-null    float64
 14  V14     697 non-null    float64
 15  V15     697 non-null    float64
 16  V16     697 non-null    float64
 17  V17     697 non-null    float64
 18  V18     697 non-null    float64
 19  V19     697 non-null    float64
 20  V20     697 non-null    float64
 21  V21     697 non-null    float64
 22  V2

In [ ]:
# Splitting into features and target
x = newdata.drop(['Class'], axis=1)
y = newdata['Class']

In [ ]:
#test train split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=42)

In [ ]:
# Define base models
base_models = [
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('lr', LogisticRegression(max_iter=1000, random_state=42))
]

In [ ]:
# Define the meta-learner
stacked_model = StackingClassifier(
    estimators=base_models,
    final_estimator=LogisticRegression()
)

In [ ]:
#Model Training
stacked_model.fit(x_train, y_train)

StackingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                               ('lr',
                                LogisticRegression(max_iter=1000,
                                                   random_state=42))],
                   final_estimator=LogisticRegression())

In [ ]:
#Prediction
prediction = stacked_model.predict(x_test)

In [ ]:
#accuracy test

accuracy = accuracy_score(y_test, prediction)

print(accuracy)

0.9714285714285714
